In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import warnings

In [18]:
df=pd.read_csv('mytrain.csv')
df_test=pd.read_csv('test.csv')

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [20]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [21]:
age=df["Age"].replace(np.nan,df['Age'].mean)
df['Age']=pd.to_numeric(df['Age'])





In [22]:
df=df.drop(['Cabin','Name',"Ticket",'PassengerId'],axis=1)
df_test=df_test.drop(['Cabin','Name',"Ticket",'PassengerId'],axis=1)

df=df.dropna()
df_test=df_test.dropna()
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
889,1,1,male,26.0,0,0,30.0000,C


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    int64  
 1   Pclass    712 non-null    int64  
 2   Sex       712 non-null    object 
 3   Age       712 non-null    float64
 4   SibSp     712 non-null    int64  
 5   Parch     712 non-null    int64  
 6   Fare      712 non-null    float64
 7   Embarked  712 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 50.1+ KB


In [24]:
df_test.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [25]:
X=df.drop('Survived',axis=1)
Y=df["Survived"]

In [26]:
num_features=X.select_dtypes(exclude="object").columns
cat_features =X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer


numaric_transform=StandardScaler()
cat_transform=OneHotEncoder()

preprocess=ColumnTransformer(
    [
    #('numerical',numaric_transform,num_features),
    ('cat_transform',cat_transform,cat_features)
    ]
)



In [27]:
X=preprocess.fit_transform(X)
test=preprocess.fit_transform(df_test)
X.shape

(712, 5)

In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,random_state=32,test_size=0.2)
#X_train.shape,X_test[:,6:7].shape

In [29]:
def evaluate_model(actual,predict):
    mse=mean_squared_error(actual,predict)
    mae=mean_absolute_error(actual,predict)
    rmse=np.sqrt(mse)
    r2_score_=r2_score(actual,predict)
    return mse,mae,r2_score_,rmse

In [30]:
model_dict={
    "catboost" : CatBoostClassifier(iterations=2000,learning_rate=0.0001),
    ""
    "SVC":SVC(C=1,kernel="poly"),
    "k-nnc":KNeighborsClassifier(),
    "DecisionTreeRegressor":DecisionTreeClassifier(),
    "AdaBoostRegressor":AdaBoostClassifier(),
    "XGBRegressor":XGBClassifier(),
    "RandomForestRegressor":RandomForestClassifier(), 
}

In [31]:
model_list =[]
r2_list=[]
i=0
for model_name in model_dict:
    model=model_dict[model_name]
    model.fit(X_train,y_train)

    y_train_pre=model.predict(X_train)
    y_test_pre=model.predict(X_test)

    model_train_mse,model_train_mae,model_train_r2_score,model_train_rmse=evaluate_model(y_train,y_train_pre)
    model_test_mse,model_test_mae,model_test_r2_score,model_test_rmse=evaluate_model(y_test,y_test_pre)
    print(list(model_dict.keys())[i])
    model_list.append(list(model_dict.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mse))
    print("- R2 Score: {:.4f}".format(model_train_r2_score))
    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mse))
    print("- R2 Score: {:.4f}".format(model_test_r2_score))
    r2_list.append(model_test_r2_score)
    
    print('='*35)
    print('\n')
    i+=1
   
    

0:	learn: 0.6931049	total: 2.09ms	remaining: 4.18s
1:	learn: 0.6930684	total: 4.04ms	remaining: 4.03s
2:	learn: 0.6930265	total: 5.49ms	remaining: 3.66s
3:	learn: 0.6929883	total: 6.97ms	remaining: 3.48s
4:	learn: 0.6929462	total: 9.46ms	remaining: 3.77s
5:	learn: 0.6929080	total: 11.5ms	remaining: 3.81s
6:	learn: 0.6928702	total: 12.9ms	remaining: 3.69s
7:	learn: 0.6928285	total: 14.6ms	remaining: 3.62s
8:	learn: 0.6927907	total: 16.1ms	remaining: 3.56s
9:	learn: 0.6927526	total: 17.6ms	remaining: 3.5s
10:	learn: 0.6927104	total: 19.3ms	remaining: 3.48s
11:	learn: 0.6926675	total: 22.7ms	remaining: 3.76s
12:	learn: 0.6926264	total: 24.4ms	remaining: 3.73s
13:	learn: 0.6925866	total: 26.4ms	remaining: 3.75s
14:	learn: 0.6925463	total: 28.3ms	remaining: 3.75s
15:	learn: 0.6925051	total: 83.2ms	remaining: 10.3s
16:	learn: 0.6924660	total: 97.4ms	remaining: 11.4s
17:	learn: 0.6924255	total: 110ms	remaining: 12.1s
18:	learn: 0.6923837	total: 111ms	remaining: 11.6s
19:	learn: 0.6923441	tota

In [32]:
tree_model=XGBClassifier()
tree_model.fit(X_train,y_train)

pre=tree_model.predict(X_test)
mse,mae,r2_score_,rmse=evaluate_model(y_test,pre)
mse,mae,r2_score_,rmse

(0.2517482517482518,
 0.2517482517482518,
 -0.05665024630541882,
 0.5017452060042545)